In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('data/train.csv')

In [ ]:
df.head()

In [ ]:
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(df, test_size = 0.1, random_state = 0)

### Missing values for categorical values

In [ ]:
features_nan = [feature for feature in df.columns if df[feature].isnull().sum() > 1 and df[feature].dtypes == 'O']

In [ ]:
for feature in features_nan:
    print(feature, np.round(df[feature].isnull().mean(), 3), '% null value')

In [ ]:
df[features_nan] = df[features_nan].fillna('Missing')

In [ ]:
for feature in features_nan:
    print(feature, np.round(df[feature].isnull().mean(), 3), '% null value')

### Missing values for numerical values

In [ ]:
numerical_features_nan = [feature for feature in df.columns if df[feature].isnull().sum() > 1 and df[feature].dtypes != 'O']

numerical_features_nan

In [ ]:
for feature in numerical_features_nan:
    median_val = df[feature].median()
    df[feature + '_nan'] = np.where(df[feature].isnull(), 1, 0)
    df[feature] = df[feature].fillna(median_val)
    

In [ ]:
df[numerical_features_nan].isnull().sum()

In [ ]:
df

### Temporal Variables

In [ ]:
year_features = [feature for feature in df.columns if 'Yr' in feature or 'Year' in feature]
year_features

In [ ]:
for feature in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    df[feature] = df['YrSold'] - df[feature]

In [ ]:
df[year_features]

### Transformations

In [ ]:
num_features = ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
for feature in num_features:
    df[feature] = np.log(df[feature])

### Handling Rare Categories
We will removecategorical variables that are present less than 1% in dataset

In [ ]:
cat_features = [feature for feature in df.columns if df[feature].dtypes == 'O']

In [ ]:
cat_features

In [ ]:
for feature in cat_features:
    temp = df.groupby(feature)['SalePrice'].count() / len(df)
    temp_df = temp[temp > 0.01].index
    df[feature] = np.where(df[feature].isin(temp_df), df[feature], 'rare_var')

In [ ]:
df.head()

In [ ]:
for feature in cat_features:
    print(feature, len(df[feature].unique()))

In [ ]:
# df = pd.get_dummies(df, drop_first = True)
# df.head()

In [ ]:
df.shape

In [ ]:
df

### Variance Threshold

In [ ]:
num_features = [feature for feature in df.columns if df[feature].dtypes != 'O']
df[num_features]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
features_scale = [feature for feature in num_features if feature not in ['Id', 'SalePrice']]

In [ ]:
data = df[num_features].copy().drop(['Id', 'SalePrice'], axis = 1)
data

In [ ]:
scaler = MinMaxScaler()
scaler.fit(data)
data = pd.DataFrame(scaler.transform(df[features_scale]), columns = features_scale)

In [ ]:
plt.bar(data.columns, data.var())
plt.show()

In [ ]:
data.var()

In [ ]:
data

In [ ]:
from sklearn.feature_selection import VarianceThreshold

var_thres = VarianceThreshold(threshold = 0.01)
var_thres.fit(data)

In [ ]:
var_thres.get_support()

In [ ]:
low_variance_cols = [col for col in data.columns if col not in data.columns[var_thres.get_support()]]
low_variance_cols

In [ ]:
df['SalePrice'].var()

In [ ]:
# df = df.drop(low_variance_cols, axis = 1)

In [ ]:
df

## Selection with correlation

In [ ]:
X = df.drop("SalePrice", axis = 1)
y = df["SalePrice"]

In [ ]:
plt.figure(figsize = (40, 40))
cor = X.corr()
sns.heatmap(cor, annot = True, cmap = plt.cm.gray)
plt.show()

In [ ]:
threshold = 0.87
corrilated_features = set()
for i in range(cor.shape[0]):
    for j in range(cor.shape[1]):
        if cor.iloc[i, j] > threshold and i != j:
            colname = cor.columns[i]
            corrilated_features.add(colname)

corrilated_features

## Mutual Info Regression

In [ ]:
from sklearn.feature_selection import mutual_info_regression

In [ ]:
X = df[num_features].copy().drop(['SalePrice', 'Id'], axis = 1)
y = df['SalePrice']

In [ ]:
mutual_info = mutual_info_regression(X, y)

In [ ]:
mutual_info

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X.columns
mutual_info.sort_values(ascending = False)

In [ ]:
mutual_info.sort_values(ascending = False).plot.bar(figsize = (15, 5))

In [ ]:
df = pd.get_dummies(df, drop_first = True)
df.head()

### Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
features_scale = [feature for feature in df.columns if feature not in ['Id', 'SalePrice']]

scaler = MinMaxScaler()
scaler.fit(df[features_scale])

In [ ]:
data = pd.DataFrame(scaler.transform(df[features_scale]), columns = features_scale)

In [ ]:
data

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression()

In [ ]:
X_train = data
y_train = df["SalePrice"]

In [ ]:
reg.fit(X_train, y_train)